In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
result = pd.read_csv("results_fix.csv",
                     names = ["strategy","congestion","demand","supply","prate","tax_cong","tax_demand",
                                            "private","revenue","cost","tax","profit","total_pax",
                                            "single_pax","shared_pax","ave_pax","ave_dist","ave_time",
                                            "public","ave_link_time","total_demand",
                                            "serviced_demand","stay_demand","serviced_rate","ave_time_offer",
                                            "ave_time_offer_s","ave_time_pickup"])

del result['public'],result['private'],result['demand'],result['congestion']
del result['ave_link_time'],result['serviced_demand'],result['stay_demand']

result.head()

,strategy,supply,prate,tax_cong,tax_demand,revenue,cost,tax,profit,total_pax,single_pax,shared_pax,ave_pax,ave_dist,ave_time,total_demand,serviced_rate,ave_time_offer,ave_time_offer_s,ave_time_pickup
0,Full,0.5,0.4,-0.000,-0.00,4620.15,1245.51,0.000,3374.64,3751,278,3473,75.02,355.86,984.84,11736,0.319870,4.25000,1.81806,7.20123
1,Full,0.5,0.4,-0.000,-0.25,4480.11,1205.05,-258.070,3533.14,3770,254,3516,75.40,344.30,978.26,11712,0.321977,4.24257,1.79130,7.25565
2,Full,0.5,0.4,-0.025,-0.00,4641.06,1233.54,-172.288,3579.81,3764,297,3467,75.28,352.44,982.58,11736,0.320808,4.24029,1.78167,7.16015
3,Full,0.5,0.4,-0.025,-0.25,4535.14,1233.12,-424.361,3726.39,3769,253,3516,75.38,352.32,983.68,11704,0.322112,4.23676,1.79708,7.16366
4,Full,0.5,0.6,-0.000,-0.00,6374.27,1224.93,0.000,5149.34,3697,323,3374,73.94,349.98,978.58,11708,0.315852,4.15613,1.76771,7.24303


In [5]:
count = 0
result_col = ['profit','ave_pax','ave_dist','ave_time','ave_time_offer','ave_time_offer_s','ave_time_pickup']
compare_stra = np.zeros((5,len(result_col)))
for supply in [0.5,1.0,1.5,2.0]:
    for prate in [0.4,0.6,0.8]:
        for tax_c in [0.0,-0.025]:
            for tax_d in [0.0,-0.25]:
                count += 1
                temp = result.loc[(result['supply'] == supply) & (result['prate'] == prate) & \
                                  (result['tax_cong'] == tax_c) & (result['tax_demand'] == tax_d),:]
                pricing_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing Opt"),result_col])
                full_temp = np.asarray(temp.loc[(result['strategy'] == "Full"),result_col])
                single_temp = np.asarray(temp.loc[(result['strategy'] == "Single"),result_col])
                shared_temp = np.asarray(temp.loc[(result['strategy'] == "Pool"),result_col])
                pricing_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing"),result_col])
                assort_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Assort Opt"),result_col])
                compare_stra[0,:] += ((pricing_opt_temp - full_temp) / pricing_opt_temp)[0,:]
                compare_stra[1,:] += ((pricing_opt_temp - single_temp) / pricing_opt_temp)[0,:]
                compare_stra[2,:] += ((pricing_opt_temp - shared_temp) / pricing_opt_temp)[0,:]
                compare_stra[3,:] += ((pricing_opt_temp - assort_opt_temp) / pricing_opt_temp)[0,:]
                compare_stra[4,:] += ((pricing_opt_temp - pricing_temp) / pricing_opt_temp)[0,:]
            
compare_stra = - compare_stra / count * 100
compare_stra

array([[ -2.87576020e+01,   2.05939742e+01,  -2.33528522e+00,
          6.53016435e+00,   1.32768133e+02,   1.03838462e+02,
          2.92927475e+00],
       [ -2.65867079e+01,  -1.42237906e+01,  -4.07225259e+00,
         -1.67260581e+00,   3.30373866e+02,   3.79005541e+02,
          1.73926504e+01],
       [ -3.48016162e+01,   2.17876511e+01,  -9.06183692e+00,
          1.78822294e+00,   2.72745581e+01,  -2.85478627e+00,
         -2.52866678e+00],
       [ -1.87746032e+01,  -2.42265638e+00,  -7.48142920e-01,
          1.99237088e-01,   1.51501874e+02,   1.50855217e+02,
          3.51267843e+00],
       [ -1.31897785e+01,   1.19304045e+01,  -2.61234580e+00,
          4.14589389e+00,   1.57514731e+02,   1.33883657e+02,
          4.86204413e+00]])